In [ ]:
import rospy 
import numpy as np 
import time 
from structure.utils.utils_structure import *
from structure.class_robot import *
from planning.class_rrt_star import RRT_STAR
from structure.class_object import OBJECT

In [ ]:
filename = "urdf/ur5e_onrobot.urdf"
pyfcl = PyFCL()
robot = ROBOT(_filename = filename) 
robot.chain.fk_chain(1)
robot.chain.fk_link_chain(1)
ctrl_num = robot.ctrl_joint_num

In [ ]:
""" 
argument: start=[x1, y1, z1], goal = [x2, y2, z2]
"""
def ExecuteConstPlan(start, goal, obs_lst):
    start_ingred = make_ik_input(target_name=['wrist_3_joint'], 
                            target_pos= [start],
                            target_rot=[[-1.57, 0, 1.57]],
                            solve_pos=[1],
                            solve_rot=[1],
                            weight_pos=1,
                            weight_rot=1,
                            disabled_joi_id=[],
                            joi_ctrl_num=ctrl_num)
    
    goal_ingred = make_ik_input(target_name=['wrist_3_joint'], 
                            target_pos= [goal],
                            target_rot=[[-1.57, 0, 1.57]],
                            solve_pos=[1],
                            solve_rot=[1],
                            weight_pos=1,
                            weight_rot=1,
                            disabled_joi_id=[],
                            joi_ctrl_num=ctrl_num)
    
    start_joi = robot.chain.get_q_from_ik(start_ingred)
    goal_joi  = robot.chain.get_q_from_ik(goal_ingred)

    if obs_lst != []: 
        obsBox_lst = []
        for obs in obs_lst: 
            obsBox = make_box(name=obs["name"], pos=obs["pos"], rot=obs["rot"], size=obs["size"]) 
            obsBox_lst.append(obsBox)
    else: 
        obsBox_lst=[]

    # Many2Many CC
    def is_in_collision(joints):
        if robot.is_collision(joints, obsBox_lst):
            return True 
        return False 
    
    rrtStar = RRT_STAR(robot, is_in_collision)
    plan, _ = rrtStar.plan(start_joi, goal_joi)
    return plan, obsBox_lst
    

In [ ]:
""" Planning """

_star    = [0.6,-0.2,0.7]
_goal    = [0.45,0.6,0.5]
_obs_lst = []
plan, obsBox_lst = ExecuteConstPlan(start=_star,goal=_goal, obs_lst=_obs_lst)

In [ ]:
""" Set objects in Unity """
table = {"pos":[0,0,0], "rot":[0,0,0], "mesh":"", "size":[1, 1, 1]}
objMesh_list = [table]

In [ ]:
print(plan)

In [ ]:
""" VISUALIZE """
opt_rendering=0
rate = rospy.Rate(10)

while not rospy.is_shutdown():
    print("start")
    while not rospy.is_shutdown(): 
        optimial_joints = plan[opt_rendering% len(plan)]
        robot.publish_opt_path(optimial_joints, r=0,g=0,b=1)
        robot.publish_obs(obsBox_lst)
        robot.publish_obj(objMesh_list)
        opt_rendering+=1
        if opt_rendering == len(plan)-1:
            break  
        rate.sleep()